In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv1 = nn.Conv2d(1, 32, 5)
        self.b1 = nn.BatchNorm2d(32)
        self.pool = nn.MaxPool2d(2, 2)
        
        self.conv2 = nn.Conv2d(32, 64, 5)
        self.b2 = nn.BatchNorm2d(64)
        
        self.fc1 = nn.Linear(64 * 4 * 4, 1024)
        self.fc2 = nn.Linear(1024, 10)

    def forward(self, x):
        # Layer 1
        #print(self.num_flat_features(x))
        x = self.pool(F.relu(self.b1(self.conv1(x))))
        #print(self.num_flat_features(x))
        # Layer 2
        x = self.pool(F.relu(self.b2(self.conv2(x))))
        
        # -- Flatten tensors
        #print(self.num_flat_features(x))
        x = x.view(-1, self.num_flat_features(x))
        
        #print(self.num_flat_features(x))
        # FC Layer 1
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        
        # FC Layer 2
        x = self.fc2(x)
        return x
    
    
    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


### Load dataset

In [7]:
import torchvision
import torchvision.transforms as transforms
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

testset = torchvision.datasets.MNIST(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=16,
                                         shuffle=False, num_workers=2)

In [5]:
# Case 1 : Load on GPU
device = torch.device("cuda")
model = Net()
model.load_state_dict(torch.load('./results/model.pth'))
model.to(device)


# Case 2 : Load on CPU
device = torch.device('cpu')
model.load_state_dict(torch.load(PATH, map_location=device))


Net(
  (conv1): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1))
  (b1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
  (b2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=1024, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=10, bias=True)
)

### Test dat

In [9]:
model.eval()

# Calculate Validation Accuracy
total_eval = 0
correct_eval = 0.0
with torch.no_grad():
    for images_eval, labels_eval in testloader:
        images_eval, labels_eval = images_eval.to(device), labels_eval.to(device)
        outputs_eval = model(images_eval)
        _, predicted_eval = torch.max(outputs_eval.data, 1)
        total_eval += labels_eval.size(0)
        correct_eval += (predicted_eval == labels_eval).sum().item() 
val_acc = correct_eval / total_eval
print('Test | Size : %6d  | ValAcc: %.3f\n' %
      (len(testloader.dataset), val_acc))

Test | Size :  10000  | ValAcc: 0.989

